In [13]:
import os
import shutil
import xml.etree.ElementTree as ET


## 1. Configuration: Directories & Files

In [14]:
# SETTINGS: Do you want to create a new GRID image
CREATE_NEW_GRID = True

In [15]:
# Grid-to-Ground scale factor
SCALE_FACTOR = 1.0000
SCALE_FACTOR = 1.0003386704  # Modify for your specific case

# Directory containing orthomosaic 
ORTHO_DIR = r"C:\Users\USFJ139860\Downloads\ORTHO_EPSG6529_20250213_Mpp0.075_VertJPEG"
ortho_file_read = os.path.join(ORTHO_DIR, "ORTHO.jpg")
aux_xml_file = ortho_file_read + ".aux.xml"

In [16]:
# Rename the image file as GRID
if CREATE_NEW_GRID:
    image_grid = os.path.splitext(ortho_file_read)[0] + "-GRID"+os.path.splitext(ortho_file_read)[1]
    shutil.copy(ortho_file_read, image_grid)
    print(f"Created:{image_grid}")
else:  # Use the original image file as GRID
    image_grid = ortho_file_read

# Create a copy of image file for GROUND
image_ground = os.path.splitext(ortho_file_read)[0] + "-GROUND"+os.path.splitext(ortho_file_read)[1]
shutil.copy(ortho_file_read, image_ground)
print(f"Created: {image_ground}")

Created:C:\Users\USFJ139860\Downloads\ORTHO_EPSG6529_20250213_Mpp0.075_VertJPEG\ORTHO-GRID.jpg
Created: C:\Users\USFJ139860\Downloads\ORTHO_EPSG6529_20250213_Mpp0.075_VertJPEG\ORTHO-GROUND.jpg


### 2. Read GeoTransform Values from AUX.XML

In [17]:
def read_geotransform(aux_file):
    if not os.path.exists(aux_file):
        raise FileNotFoundError(f"Error: AUX.XML file not found at {aux_file}")
    
    tree = ET.parse(aux_file)
    root = tree.getroot()
    geotransform = root.find(".//GeoTransform")
    
    if geotransform is None or not geotransform.text:
        raise ValueError("Error: <GeoTransform> not found in AUX.XML file")
    
    values = list(map(float, geotransform.text.split(',')))
    if len(values) != 6:
        raise ValueError("Error: Invalid number of values in <GeoTransform>")
    
    return values


# 3. Extract geotransform values


In [18]:
geo_values = read_geotransform(aux_xml_file)
x0, gsd_x, theta_x, y0, theta_y, gsd_y = geo_values

# 4. Generate JGW Files

In [19]:
grid_jgw = os.path.splitext(image_grid)[0] +".jgw"
ground_jgw = os.path.splitext(image_ground)[0] +".jgw" 
scale_factor_jgw = os.path.splitext(ortho_file_read)[0] +"-sf-grid-to-ground.txt" 

In [20]:
with open(grid_jgw, 'w') as f:
    f.write(f"{gsd_x}\n{theta_y}\n{theta_x}\n{gsd_y}\n{x0}\n{y0}\n")
print(f"File created: {grid_jgw}")


with open(scale_factor_jgw, 'w') as f:
    f.write(f"Grid-to-Ground Scale Factor:\n{SCALE_FACTOR}\n")
print(f"File created: {scale_factor_jgw}")



# Apply scale factor
gsd_x *= SCALE_FACTOR
gsd_y *= SCALE_FACTOR
x0 *= SCALE_FACTOR
y0 *= SCALE_FACTOR


with open(ground_jgw, 'w') as f:
    f.write(f"{gsd_x}\n{theta_y}\n{theta_x}\n{gsd_y}\n{x0}\n{y0}\n")
print(f"File created: {ground_jgw}")

File created: C:\Users\USFJ139860\Downloads\ORTHO_EPSG6529_20250213_Mpp0.075_VertJPEG\ORTHO-GRID.jgw
File created: C:\Users\USFJ139860\Downloads\ORTHO_EPSG6529_20250213_Mpp0.075_VertJPEG\ORTHO-sf-grid-to-ground.txt
File created: C:\Users\USFJ139860\Downloads\ORTHO_EPSG6529_20250213_Mpp0.075_VertJPEG\ORTHO-GROUND.jgw
